# Multi-Chain to Generate source code


In [40]:
import os
from uuid import uuid4

In [32]:
# 导入必要的库
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

## Create chain for Python language

In [42]:
template_string="""
You are Raymond Hetting, an expert python programmer, 
well versed in meta-programming and elegant, concise and short but well documented code. 
You follow the PEP8 style guide. 

Task: 
1.Write me a Python code to fullfil below requirements.
2.Only response me the code generated, NO other context or descriptions.

[requirements]
{code_requirement}
"""

prompt=ChatPromptTemplate.from_template(template_string)

llm = ChatOpenAI(model="gpt-4o-mini")
python_chain = (
    prompt
    |llm
    |StrOutputParser()
)

for chunk in python_chain.stream("write me a quick sort code"):
    print(chunk, end="", flush=True)
    

```python
def quicksort(arr):
    """Sorts an array using the Quick Sort algorithm."""
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# Example usage:
# sorted_array = quicksort([3, 6, 8, 10, 1, 2, 1])
```

## Create chain for Java Code

In [43]:
Java_template_string="""
You are Chris Lattner, an expert Java programmer, 
well versed in meta-programming and elegant, concise and short but well documented code. 
You follow the Java 18 style guide. 

Task: 
1.Write me a Java code to fullfil below requirements.
2.Only response me the code generated, NO other context or descriptions.

[requirements]
{code_requirement}
"""

Java_prompt=ChatPromptTemplate.from_template(Java_template_string)

Java_llm = ChatOpenAI(model="gpt-4o-mini")
Java_chain = (
    Java_prompt
    |Java_llm
    |StrOutputParser()
)

for chunk in Java_chain.stream("write me a quick sort code"):
    print(chunk, end="", flush=True)

```java
import java.util.Arrays;

public class QuickSort {

    public static void main(String[] args) {
        int[] array = {10, 7, 8, 9, 1, 5};
        quickSort(array, 0, array.length - 1);
        System.out.println("Sorted array: " + Arrays.toString(array));
    }

    private static void quickSort(int[] arr, int low, int high) {
        if (low < high) {
            int pivotIndex = partition(arr, low, high);
            quickSort(arr, low, pivotIndex - 1); // Recursively sort left subarray
            quickSort(arr, pivotIndex + 1, high); // Recursively sort right subarray
        }
    }

    private static int partition(int[] arr, int low, int high) {
        int pivot = arr[high]; // Choose the rightmost element as pivot
        int i = (low - 1); // Index of the smaller element

        for (int j = low; j < high; j++) {
            if (arr[j] <= pivot) {
                i++; // Increment index of smaller element
                swap(arr, i, j);
            }
        }
   

## Combine two

In [44]:
final_prompt = ChatPromptTemplate.from_template("Given Python code:\n{result1} \n and Java code:\n{result2} \n You should only print the origin foramtted code for me.")

gen_code_chain=(
    {"code_requirement":RunnablePassthrough()}
    |{
        "result1":python_chain,
        "result2": Java_chain
    }
    |final_prompt    
    |ChatOpenAI(model="gpt-4o-mini")
    |StrOutputParser()
)


for chunk in gen_code_chain.stream("write me a quick sort code"):
    print(chunk, end="", flush=True)
    

```python
def quick_sort(arr):
    """Sorts an array using the quick sort algorithm."""
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# Example usage
# sorted_array = quick_sort([3, 6, 8, 10, 1, 2, 1])
# print(sorted_array)
```

```java
import java.util.Arrays;

public class QuickSort {
    
    public static void main(String[] args) {
        int[] array = {10, 7, 8, 9, 1, 5};
        quickSort(array, 0, array.length - 1);
        System.out.println("Sorted array: " + Arrays.toString(array));
    }

    // Main method to perform quick sort
    public static void quickSort(int[] arr, int low, int high) {
        if (low < high) {
            int pi = partition(arr, low, high);
            quickSort(arr, low, pi - 1); // Recursively sort elements before partition
            quic